In [5]:
import os
import shutil
import torch

In [6]:
from include.utils import dividir_dataset, mover_labels_imagens
from ultralytics import YOLO

In [3]:
class ModeloYolo:
    def __init__(self, modelo_path, output_dir):
        self.modelo = YOLO(modelo_path)
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

    def treinar(self, dataset_path, epochs_treinamento=10, batch_size=16, imgsz=640, lr=0.001, patience=3):
        print("Iniciando o treinamento...")
        
        best_loss = float('inf')
        epochs_no_improvement = 0
        start_epoch = 0
        
        checkpoint_path = os.path.join(self.output_dir, 'checkpoint.pt')

        # Carregar checkpoint se existir
        if os.path.exists(checkpoint_path):
            print("Checkpoint encontrado. Carregando...")
            self.modelo = YOLO(checkpoint_path)
            with open(os.path.join(self.output_dir, 'checkpoint_info.txt'), 'r') as f:
                start_epoch = int(f.read())
            print(f"Retomando do epoch {start_epoch + 1}")
    
        for epoch in range(start_epoch, epochs):
            print(f"\nÉpoca {epoch + 1}/{epochs}")
    
            results = self.modelo.train(
                data=dataset_path,
                imgsz=imgsz,
                batch=batch_size,
                epochs=epochs_treinamento,
                lr0=lr,
                augment=True,
                verbose=False,
                resume=False,
                device="cpu"
            )
    
            metrics = results.results_dict
            current_loss = metrics.get('train_loss', float('inf'))
    
            # Salvar o modelo se a perda melhorar
            if current_loss < best_loss:
                best_loss = current_loss
                epochs_no_improvement = 0
                best_model_path = os.path.join(self.output_dir, 'modelo_treinado.pt')
                self.modelo.save(best_model_path)  # Salvar o modelo completo
                print(f"Modelo melhorado salvo em: {best_model_path} com perda: {current_loss:.4f}")
            else:
                epochs_no_improvement += 1
                print(f"Sem melhora na perda ({current_loss:.4f}), paciência: {epochs_no_improvement}/{patience}")
    
            # Salvar checkpoint do modelo
            checkpoint_path = os.path.join(self.output_dir, f'checkpoint_epoch_{epoch}.pt')
            self.modelo.save(checkpoint_path)
            with open(os.path.join(self.output_dir, 'checkpoint_info.txt'), 'w') as f:
                f.write(str(epoch))
            print(f"Checkpoint salvo em: {checkpoint_path}")
    
            if epochs_no_improvement >= patience:
                print("Parada antecipada ativada. Treinamento interrompido.")
                break
    
        final_model_path = os.path.join(self.output_dir, 'modelo_treinado.pt')
        self.modelo.save(final_model_path)  # Salvar o modelo completo
        print(f"Modelo final salvo em: {final_model_path}")

        print("Treinamento concluído.")

In [4]:
current_dir = os.getcwd()

caminho_imagens = os.path.join(current_dir, 'arquivo', 'imagens')  # Pasta onde as imagens estão
caminho_modelo = os.path.join(current_dir, 'modelo')  # Diretório onde o modelo treinado será salvo
caminho_test = os.path.join(current_dir, 'treinamento', 'test')  # Pasta para as imagens de teste
caminho_treino = os.path.join(current_dir, 'treinamento', 'train')  # Pasta para as imagens de treino
caminho_val = os.path.join(current_dir, 'treinamento', 'val')  # Pasta para as imagens de validação
caminho_yaml = os.path.join(current_dir, 'dataset.yaml')  # Caminho para o arquivo YAML de configuração do dataset

# Dividir o dataset
dividir_dataset(caminho_imagens, caminho_treino, caminho_val, caminho_test)

# Inicializar e treinar o modelo
modelo = os.path.join(caminho_modelo,'yolo11s.pt')
classe_modelo = ModeloYolo(modelo, caminho_modelo)
classe_modelo.treinar(caminho_yaml)

Divisão do dataset concluída: 276 imagens para treino, 34 para validação, 35 para teste.
Iniciando o treinamento...

Época 1/10
New https://pypi.org/project/ultralytics/8.3.70 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.68  Python-3.12.4 torch-2.5.0.dev20240827+cu118 CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\modelo\yolo11s.pt, data=C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=

train: Scanning C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens.cache... 345 images, 329 backgrounds, 0 corrupt: 100%|██████████| 674/674 [00:00<?, ?it/s]
val: Scanning C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens.cache... 345 images, 329 backgrounds, 0 corrupt: 100%|██████████| 674/674 [00:00<?, ?it/s]


Plotting labels to runs\detect\train3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.151      3.716      1.555          7        640: 100%|██████████| 43/43 [2:05:12<00:00, 174.71s/it]  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  91%|█████████ | 20/22 [04:36<00:26, 13.49s/it]

WARNING  NMS time limit 3.600s exceeded


...

                   all        674        385      0.145      0.242     0.0804     0.0304
Speed: 3.5ms preprocess, 675.1ms inference, 0.0ms loss, 116.3ms postprocess per image
Results saved to runs\detect\train3
Sem melhora na perda (inf), paciência: 1/3
Checkpoint salvo em: C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\modelo\checkpoint_epoch_0.pt

Época 2/10
New https://pypi.org/project/ultralytics/8.3.70 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.68  Python-3.12.4 torch-2.5.0.dev20240827+cu118 CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\modelo\yolo11s.pt, data=C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=0, project=None, name=train32, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, s

train: Scanning C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens.cache... 345 images, 329 backgrounds, 0 corrupt: 100%|██████████| 674/674 [00:00<?, ?it/s]
val: Scanning C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens.cache... 345 images, 329 backgrounds, 0 corrupt: 100%|██████████| 674/674 [00:00<?, ?it/s]


Plotting labels to runs\detect\train32\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train32
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      1.119      1.817      1.491         22        640:  70%|██████▉   | 30/43 [1:37:18<42:09, 194.60s/it]  
Exception ignored in: <function _xla_gc_callback at 0x00000208974EBD80>
Traceback (most recent call last):
  File "C:\Users\luiz.santos\AppData\Local\anaconda3\Lib\site-packages\jax\_src\lib\__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

In [8]:
current_dir = os.getcwd()

caminho_imagens = os.path.join(current_dir, 'arquivo', 'imagens')  # Pasta onde as imagens estão
caminho_modelo = os.path.join(current_dir, 'modelo')  # Diretório onde o modelo treinado será salvo

# Avaliando modelo
caminho_modelo_treinado = os.path.join(caminho_modelo,'modelo_treinado.pt')  # Diretório do modelo treinado
modelo = YOLO(caminho_modelo_treinado)
modelo.val()

caminho_imagem_teste = os.path.join(caminho_imagens, 'img_425.jpg')
results = modelo(caminho_imagem_teste)

caminho_imagem_teste = os.path.join(caminho_imagens, 'img_500.jpg')
results = modelo(caminho_imagem_teste)

caminho_imagem_teste = os.path.join(caminho_imagens, 'img_650.jpg')
results = modelo(caminho_imagem_teste)

Ultralytics 8.3.68  Python-3.12.4 torch-2.5.0.dev20240827+cu118 CPU (12th Gen Intel Core(TM) i5-1235U)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens.cache... 345 images, 329 backgrounds, 0 corrupt: 100%|██████████| 674/674 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  21%|██        | 9/43 [02:18<06:32, 11.55s/it]

WARNING  NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [06:18<00:00,  8.80s/it]


                   all        674        385      0.361      0.642      0.429      0.201
Speed: 4.1ms preprocess, 494.7ms inference, 0.0ms loss, 11.5ms postprocess per image
Results saved to runs\detect\val3

image 1/1 C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens\img_425.jpg: 480x640 1 objetos_cortantes, 643.0ms
Speed: 33.1ms preprocess, 643.0ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens\img_500.jpg: 480x640 1 objetos_cortantes, 396.7ms
Speed: 8.3ms preprocess, 396.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\luiz.santos\Documents\FIAP Grupo\Hackathon\arquivo\imagens\img_650.jpg: 480x640 3 objetos_cortantess, 227.5ms
Speed: 3.0ms preprocess, 227.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
